# RAGAS TEST
## 1. VectorDB 비교
#### FAISS-cpu vs ChromaDB
#### 선정 이유
- 접근성
- 로컬 저장
- 성능
- 커뮤니티

In [1]:
!pip install langchain langchain-openai langchain-chroma langchain-community openai faiss-cpu python-docx dotenv rank_bm25 ragas transformers sentence_transformers

  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached pandas-2.3.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.2 MB ? eta -:--:--
   ------------------------------------ --- 10.2/11.2 MB 49.1 MB/s eta 0:00:01
   ---------------------------------------- 11.2/11.2 MB 46.6 MB/s eta 0:00:00
Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl (308 kB)
   ------------------------

### 환경 변수

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = os.getenv("OPENAI_API_BASE")

print("done")

done


## RAGAS 평가 코드 정의

In [16]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)
import json
from pprint import pprint

QA_set = []

try:
    if os.path.exists("ground_truths.json"):
        with open("ground_truths.json", "r", encoding="utf-8") as f:
            ground_truths = json.load(f)
        print("✅ GT 로드 완료")
    else:
        raise FileNotFoundError

    for doc in ground_truths:
        QA_set.append(doc['qa_pairs'])
    else:
        print("Q&A set 생성 완료")
except FileNotFoundError:
    print("❌ GT 파일이 존재하지 않음")


✅ GT 로드 완료
Q&A set 생성 완료
